# 编译器和解释器

In [1]:
def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

print(fancy_func(1, 2, 3, 4))

10


In [2]:
def add_():
    return '''
def add(a, b):
    return a + b
'''

def fancy_func_():
    return '''
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
'''

def evoke_():
    return add_() + fancy_func_() + 'print(fancy_func(1, 2, 3, 4))'

prog = evoke_()
print(prog)
y = compile(prog, '', 'exec')
exec(y)


def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
print(fancy_func(1, 2, 3, 4))
10


In [ ]:
import torch
from torch import nn


# 生产网络的工厂模式
def get_net():
    net = nn.Sequential(nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2))
    return net

x = torch.randn(size=(1, 512))
net = get_net()
net(x)

tensor([[ 0.2698, -0.0975]], grad_fn=<AddmmBackward0>)

In [4]:
net = torch.jit.script(net)
net(x)

tensor([[ 0.2698, -0.0975]], grad_fn=<AddmmBackward0>)

In [6]:
import time

class Benchmark:
    """
    用于测量代码块运行时间的基准测试类。
    
    使用方法:
    with Benchmark('一些描述'):
        # 需要测量的代码
        ...
    """
    def __init__(self, description='Done'):
        """
        构造函数。
        
        参数:
        description (str): 对被测量代码块的描述，将显示在最终输出中。
        """
        self.description = description

    def __enter__(self):
        """在进入 with 块时被调用，记录开始时间。"""
        # 使用 time.perf_counter() 来记录高精度的开始时间
        self.start_time = time.perf_counter()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        """在退出 with 块时被调用，计算并打印所用时间。"""
        # 记录结束时间
        end_time = time.perf_counter()
        # 计算时间差并打印
        elapsed_time = end_time - self.start_time
        print(f'{self.description}: {elapsed_time:.4f} sec')

In [7]:
net = get_net()
with Benchmark('无torchscript'):
    for i in range(1000): net(x)

net = torch.jit.script(net)
with Benchmark('有torchscript'):
    for i in range(1000): net(x)

无torchscript: 0.0792 sec
有torchscript: 0.0321 sec


In [ ]:
net.save('my_mlp')
!ls -lh my_mlp*

# -rw-r--r--@ 1 xuzhiyuan  staff   651K Jul  3 15:08 my_mlp

-rw-r--r--@ 1 xuzhiyuan  staff   651K Jul  3 15:08 my_mlp


# 异步计算

In [9]:
import os
import subprocess
import numpy
import torch
from torch import nn

In [10]:
# GPU计算热身
device = "mps"
a = torch.randn(size=(1000, 1000), device=device)
b = torch.mm(a, a)

with Benchmark('numpy'):
    for _ in range(10):
        a = numpy.random.normal(size=(1000, 1000))
        b = numpy.dot(a, a)

with Benchmark('torch'):
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)

numpy: 0.9969 sec
torch: 0.0041 sec


In [13]:
with Benchmark():
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)
    torch.mps.synchronize()

Done: 0.0590 sec


In [15]:
x = torch.ones((1, 2), device=device)
y = torch.ones((1, 2), device=device)
z = x * y + 2
z

tensor([[3., 3.]], device='mps:0')

In [16]:
import torch
from torch import nn
from torch.nn import functional as F